In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

# Creation of a churn model for the company and new processes for churn reduction

## Table of Contents
1. [Project overview](#Project_overview)
  1. [Company background](#Company_background) 
  1. [Project background](#Project_background) 
  1. [Project goal](#Project_goal) 
  1. [Project objectives](#Project_objectives) 
1. [Theoretical background](#Theoretical_background)  .
  1. [Sample imbalance](#Sample_imbalance)
  1. [Time-aware cross-validation](#Time_CV)
1. [Data](#Data)
  1. [Churn definition](#Churn_definition) 
  1. [Data overview](#Data_overview) 
  1. [Data preparation](#Data_preparation) 
1. [Exploratory data analysis](#EDA) 


<a id="Project_overview"></a>
## Project overview

<a id="Company_background"></a>
### Company background

TELE2 is one of the most popular telecom operators in Russia. It was established in 2003 and now operates in 65 regions of Russian Federation, providing services in high-speed internet connection of 3G/4G standard. The main track of company’s development is creation of products with the best available price-quality ratio. It has significant success among clients and has become the fastest growing operator by average year revenue from services. To sum up, current approach allowed the company to not only attract the price-sensitive customer segment, but also capture the most growing segment of digital clients, who use the data (internet access).

<a id="Project_background"></a>
### Project background

To increase revenue and profit, mobile operators need to increase their user base. There are two ways to achieve this. Companies can attract new users or they can reduce the outflow of existing users. At the same time, it is true for the telecom industry in Russia nowadays that retaining one user is cheaper than attracting one user. However, user retention measures are quite expensive and the company cannot carry out user retention measures for all customers. That is why TELE2 needs to understand exactly which customers are going to change their mobile operator.

Creating customer churn models is one of the most pressing data science challenges for the industry. Timely identification of users who are going to change their mobile operator allows company to start activities aimed at retaining such customers on time. That is why the project is of great practical value for TELE2

There are two important points in building customer retention processes in the company. Firstly, retention of the customer does not happen instantly, which means that company needs to start retention measures before the user decides to change mobile operator. The second important point is that some users bring more profit to the company than others, which means TELE2 needs to differentiate customer retention measures and spend more resources on retention of customers, which are important for the company.

<a id="Project_goal"></a>
### Project goal

The goal of the project is to create a churn model for the company and suggest processes for churn reduction.

<a id="Project_objectives"></a>
### Project objectives

1. Define churn.
1. Choose type of the model (regression, binary classification, multiclass classification)
1. Determine how much time is required for customer retention measures and based on this determine the prediction horizon for the model.
1. Choose quality metric.
1. Build a model on historical data.
1. Suggest an experiment design for testing model on real users.


<a id="Theoretical_background"></a>
## Theoretical background

<a id="Sample_imbalance"></a>
### Sample imbalance

TODO

<a id="Time_CV"></a>
### Time-aware cross-validation

TODO

<a id="Data"></a>
## Data

<a id="Churn_definition"></a>
### Churn definition

Tele2 defines churned customer as customer who was not active for 90 days. The company needs 5 days to conduct customer retention measures. It is clear that such measures should be carried out before the client ceased to use the services of a mobile operator. Thus, the curn model goal is to determine the likelihood that the user will make his last flash activity in the next 7-14 days.  
![Definition](img/churn_definition.png)  

Thus, the chronology of customer retention measures will be following. Suppose on March 3 we are scoring all clients (scoring_day). At this moment, we data available as on March 1 (report_day). Then our model should predict the probability that the client will make the last flash activity in the period from March 8 to March 14. Now the company has time from March 3 to March 8 to try to contact the user, offer him/her a discount, special conditions and so on.  

![Chronology](img/churn_chronology.png)

<a id="Data_overview"></a>
### Data overview

In [2]:
data = pd.read_csv('churn_data.zip')
data.head(n=10)

,report_date,label,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,client_id
0,2018-11-01,0,1.3700,0.272727,2844,1,7,0,97.5700,15,...,0.0000,997.25,36.0,1.0,NaN,2.0,3.0,4.0,0.00,iac7tbnxs4
1,2019-01-01,0,1.4719,6.857143,700,30,3,0,400.9999,29,...,398.9999,3084.22,15.0,1.0,28.0,1.0,12.0,12.0,0.55,nq1gzprvl4
2,2018-11-01,0,-1.2900,117.727273,2232,0,9,0,25.5000,0,...,3.0000,88.00,2.0,1.0,1.0,8.0,3.0,8.0,0.00,ujfhavmvea
3,2018-11-01,0,8.7723,0.045455,704,17,3,0,354.9999,22,...,89.9999,2556.00,29.0,2.0,4.0,1.0,3.0,4.0,0.00,s6p0vg7kcr
4,2018-10-01,0,249.3617,15.250000,1057,30,4,0,398.9999,4,...,398.9999,2680.00,8.0,0.0,12.0,2.0,12.0,12.0,0.00,3b7lxt5rsk
5,2018-12-01,0,57.6500,1.250000,4812,30,9,0,270.0000,25,...,200.0000,1800.00,13.0,1.0,15.0,1.0,2.0,2.0,0.00,rr9twyst2j
6,2018-11-01,0,-313.2998,11.318182,1378,30,22,0,329.0000,13,...,299.0000,3520.58,14.0,2.0,13.0,1.0,12.0,16.0,0.00,vn82a3f903
7,2019-01-01,0,0.2463,19.333333,509,6,6,0,56.7995,20,...,7.9999,1007.00,69.0,1.0,1.0,1.0,2.0,2.0,0.00,ja4f8k2pzu
8,2019-01-01,1,-32.3299,1.714286,2279,0,7,0,96.8500,2,...,0.0000,1613.94,28.0,2.0,NaN,1.0,2.0,10.0,0.00,datw3tw6d2
9,2018-10-01,0,77.5500,1.458333,3009,0,2,0,43.8800,20,...,NaN,251.00,4.0,1.0,NaN,6.0,0.0,36.0,0.00,uiw9672bd4


,report_date,label,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,...,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,client_id
0,2018-11-01,0,1.3700,0.272727,2844,1,7,0,97.5700,15,...,0.0000,997.25,36.0,1.0,NaN,2.0,3.0,4.0,0.00,iac7tbnxs4
1,2019-01-01,0,1.4719,6.857143,700,30,3,0,400.9999,29,...,398.9999,3084.22,15.0,1.0,28.0,1.0,12.0,12.0,0.55,nq1gzprvl4
2,2018-11-01,0,-1.2900,117.727273,2232,0,9,0,25.5000,0,...,3.0000,88.00,2.0,1.0,1.0,8.0,3.0,8.0,0.00,ujfhavmvea
3,2018-11-01,0,8.7723,0.045455,704,17,3,0,354.9999,22,...,89.9999,2556.00,29.0,2.0,4.0,1.0,3.0,4.0,0.00,s6p0vg7kcr
4,2018-10-01,0,249.3617,15.250000,1057,30,4,0,398.9999,4,...,398.9999,2680.00,8.0,0.0,12.0,2.0,12.0,12.0,0.00,3b7lxt5rsk
5,2018-12-01,0,57.6500,1.250000,4812,30,9,0,270.0000,25,...,200.0000,1800.00,13.0,1.0,15.0,1.0,2.0,2.0,0.00,rr9twyst2j
6,2018-11-01,0,-313.2998,11.318182,1378,30,22,0,329.0000,13,...,299.0000,3520.58,14.0,2.0,13.0,1.0,12.0,16.0,0.00,vn82a3f903
7,2019-01-01,0,0.2463,19.333333,509,6,6,0,56.7995,20,...,7.9999,1007.00,69.0,1.0,1.0,1.0,2.0,2.0,0.00,ja4f8k2pzu
8,2019-01-01,1,-32.3299,1.714286,2279,0,7,0,96.8500,2,...,0.0000,1613.94,28.0,2.0,NaN,1.0,2.0,10.0,0.00,datw3tw6d2
9,2018-10-01,0,77.5500,1.458333,3009,0,2,0,43.8800,20,...,NaN,251.00,4.0,1.0,NaN,6.0,0.0,36.0,0.00,uiw9672bd4


In [3]:
print ("# Rows:", data.shape[0])
print ("# Columns:", data.shape[1])

# Rows: 280000
# Columns: 49
# Rows: 280000
# Columns: 49


In [4]:
pd.concat([pd.Series(data.dtypes, name='data_type'),
           data.describe().transpose(),
           pd.Series(data.nunique(), name='unique_values'), 
           pd.Series(data.isnull().sum(), name='missing_values'),
           ], axis=1)



,data_type,count,mean,std,min,25%,50%,75%,max,unique_values,missing_values
report_date,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
label,int64,280000.0,0.142857,0.349928,0.000000,0.000000,0.000000,0.000000,1.000000,2,0
feat_1,float64,280000.0,23.893187,181.351308,-11573.226900,0.050800,3.450250,33.890000,18773.840000,126640,0
feat_2,float64,280000.0,8.427203,23.424582,-1.000000,0.227273,4.333333,11.833333,2953.363636,7358,0
feat_3,int64,280000.0,1881.572993,1541.856057,61.000000,519.000000,1412.000000,3094.000000,17897.000000,6102,0
feat_4,int64,280000.0,13.243425,13.445184,0.000000,0.000000,8.000000,29.000000,30.000000,31,0
feat_5,int64,280000.0,7.022521,9.679502,0.000000,3.000000,5.000000,9.000000,2785.000000,187,0
feat_6,int64,280000.0,0.834732,2.219580,0.000000,0.000000,0.000000,0.000000,13.000000,14,0
feat_7,float64,280000.0,278.751168,266.851525,0.020000,99.000000,239.975000,372.019925,9272.209700,61993,0
feat_8,int64,280000.0,5.725643,8.435074,0.000000,0.000000,0.000000,10.000000,30.000000,31,0


,data_type,count,mean,std,min,25%,50%,75%,max,unique_values,missing_values
report_date,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0
label,int64,280000.0,0.142857,0.349928,0.000000,0.000000,0.000000,0.000000,1.000000,2,0
feat_1,float64,280000.0,23.893187,181.351308,-11573.226900,0.050800,3.450250,33.890000,18773.840000,126640,0
feat_2,float64,280000.0,8.427203,23.424582,-1.000000,0.227273,4.333333,11.833333,2953.363636,7358,0
feat_3,int64,280000.0,1881.572993,1541.856057,61.000000,519.000000,1412.000000,3094.000000,17897.000000,6102,0
feat_4,int64,280000.0,13.243425,13.445184,0.000000,0.000000,8.000000,29.000000,30.000000,31,0
feat_5,int64,280000.0,7.022521,9.679502,0.000000,3.000000,5.000000,9.000000,2785.000000,187,0
feat_6,int64,280000.0,0.834732,2.219580,0.000000,0.000000,0.000000,0.000000,13.000000,14,0
feat_7,float64,280000.0,278.751168,266.851525,0.020000,99.000000,239.975000,372.019925,9272.209700,61993,0
feat_8,int64,280000.0,5.725643,8.435074,0.000000,0.000000,0.000000,10.000000,30.000000,31,0


In [5]:
print('Distribution of report dates:')
print(data['report_date'].value_counts().sort_index())

Distribution of report dates:
2018-10-01    70000
2018-11-01    70000
2018-12-01    70000
2019-01-01    70000
Name: report_date, dtype: int64
Distribution of report dates:
2018-10-01    70000
2018-11-01    70000
2018-12-01    70000
2019-01-01    70000
Name: report_date, dtype: int64


In [6]:
print('Share of target class for different report dates:')
data.groupby(['report_date'])['label'].mean()

Share of target class for different report dates:
Share of target class for different report dates:


report_date
2018-10-01    0.142857
2018-11-01    0.142857
2018-12-01    0.142857
2019-01-01    0.142857
Name: label, dtype: float64

report_date
2018-10-01    0.142857
2018-11-01    0.142857
2018-12-01    0.142857
2019-01-01    0.142857
Name: label, dtype: float64

Results of the data overview:
- There are 280,000 records in the dataset provided by the company. Each row of the dataset is describing Tele2 customer.
- For each entry there is a class label. Class “1” means that the client churned, class “0” means that the client remained with the company.
- Each entry describes the customer by 46 characteristics. The company did not provide a description of the features. This means that we will use the names of features like  “feat_1”, “feat_2” and so on.
- Four reporting dates are presented in the data: October 1, 2018, November 1, 2018, December 1, 2018 and January 1, 2019. For each date, there are 70,000 records.
- All client_id are unique, which means that there is no intersection in clients across different reporting dates.
- The share of churned clients is the same in all reporting periods. It is just over 14%.


<a id="Data_preparation"></a>
### Data preparation

Now let's create a holdout set. Typically, the holdout set is approximately 20% of the available data. In our case, it makes sense to raise this figure to 25% and use as a holdout set all the data for the latest available date, i.e. for January 2019. This way we can simulate the real work of the model. It will be trained and validated on historical data and then launched on data that came later. 

We will save our holdout set in a separate file. Thus, we guarantee that this data will definitely not be used during training the model.

In [7]:
# data['report_date']= pd.to_datetime(data['report_date']) 

In [8]:
holdout = data[data['report_date'] == '2019-01-01']
holdout.to_csv('holdout.zip', compression='zip', index=False)
print("The size of holdout set is:", holdout.shape)

The size of holdout set is: (70000, 49)
The size of holdout set is: (70000, 49)


In [9]:
train = data[data['report_date'] != '2019-01-01']
train.to_csv('train.zip', compression='zip', index=False)
print("The size of training set is:", train.shape)

The size of training set is: (210000, 49)
The size of training set is: (210000, 49)


In [10]:
del data
del holdout
del train

<a id="EDA"></a>
### Exploratory data analysis

In [11]:
# plt.figure(figsize=(10, 115))
# 
# idx = 0
# for column in train.columns:
#     if column in ['label', 'report_date', 'client_id']:
#         continue
#     plt.subplot(23, 2, idx + 1)
#     sns.violinplot(x='label', y=column, data=train, cut=3)
#     idx += 1


<a id="Baseline_model"></a>
### Baseline model

In [12]:
X = pd.read_csv('train.zip').drop(['client_id', 'report_date'], axis=1)
y = X.pop('label')

X = X.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
model = Sequential()
model.add(Dense(64, input_dim=46, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0)]

In [15]:
class_weight = {0: 0.5,
                1: 2}

In [16]:
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          class_weight = class_weight,
          callbacks=callbacks,
          epochs=1000,
          #batch_size=256, 
          verbose=2)

Train on 157500 samples, validate on 52500 samples
Epoch 1/1000
 - 15s - loss: 5.9014 - accuracy: 0.7064 - val_loss: 0.5718 - val_accuracy: 0.8586
Epoch 2/1000
 - 14s - loss: 0.4859 - accuracy: 0.8477 - val_loss: 0.5597 - val_accuracy: 0.8585
Epoch 3/1000
 - 14s - loss: 0.4610 - accuracy: 0.8532 - val_loss: 0.5112 - val_accuracy: 0.8586
Epoch 4/1000
 - 14s - loss: 0.4403 - accuracy: 0.7944 - val_loss: 0.4274 - val_accuracy: 0.8624
Epoch 5/1000
 - 13s - loss: 0.4189 - accuracy: 0.8289 - val_loss: 0.4291 - val_accuracy: 0.8518
Epoch 6/1000
 - 14s - loss: 0.4024 - accuracy: 0.8102 - val_loss: 0.4705 - val_accuracy: 0.8450
Epoch 7/1000
 - 14s - loss: 0.3981 - accuracy: 0.8061 - val_loss: 0.4039 - val_accuracy: 0.8463
Epoch 8/1000
 - 14s - loss: 0.3959 - accuracy: 0.7912 - val_loss: 0.4720 - val_accuracy: 0.8030
Epoch 9/1000
 - 13s - loss: 0.3958 - accuracy: 0.7912 - val_loss: 0.4624 - val_accuracy: 0.8370
Epoch 10/1000
 - 13s - loss: 0.3979 - accuracy: 0.7919 - val_loss: 0.4420 - val_accur

In [17]:
model.save('baseline.h5') 

In [18]:
y_pred = model.predict_classes(X_test)

In [19]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

Confusion Matrix
[[38279  6794]
 [ 2638  4789]]
Confusion Matrix
[[37549  7524]
 [ 2587  4840]]


In [20]:
print('Classification Report')
print(classification_report(y_test, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.85      0.89     45073
           1       0.41      0.64      0.50      7427

    accuracy                           0.82     52500
   macro avg       0.67      0.75      0.70     52500
weighted avg       0.86      0.82      0.84     52500

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.83      0.88     45073
           1       0.39      0.65      0.49      7427

    accuracy                           0.81     52500
   macro avg       0.66      0.74      0.69     52500
weighted avg       0.86      0.81      0.83     52500

